In [1]:
%matplotlib notebook
import flux

Missing flux value for: GLEAM J202818-254302 at frequency: 204 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 212 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 220 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 227 MHz.
Missing spectral index for: GLEAM J202818-254302


//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp

In [3]:
arbitrary_source = flux.catalog.obj_catalog[999]
arb_ra = arbitrary_source.ra_angle
arb_dec = arbitrary_source.dec_angle
J = flux.stokes.J_matrix(arb_ra, arb_dec)
J.shape

(1, 2, 2)

In [7]:
arb_ra = flux.rot.get_lst()
arb_dec = np.radians(flux.rot.hera_lat)
J = flux.stokes.J_matrix(arb_ra, arb_dec)
J

array([[[ 2.80981413e-05+3.16798088e-05j,
          8.12625503e-01-5.82786230e-01j],
        [ 8.12625503e-01-5.82786230e-01j,
         -2.80981411e-05-3.16798085e-05j]]])

In [ ]:
Am = flux.stokes.A_matrix(arb_ra, arb_dec)
print(Am)
Am.shape

In [ ]:
Am[0][0][0]

In [ ]:
# Display available antennae
print(flux.demo.active_ants)

In [ ]:
# we sample three sources almost at random, and 2/3 are zero

print(arbitrary_source)
print(flux.vis.visibility(2, 11, arbitrary_source))

print(flux.vis.visibility(36, 37, flux.catalog.obj_catalog[981]))
print(flux.vis.visibility(88, 121, flux.catalog.obj_catalog[134]))
# also, would we not expect the first term to dominate? And yet it is not even the largest
    # of numbers all with magnitudes less than one

In [ ]:
# proposal for version 7: change J_matrix to include local sidereal time

def J_matrix_7(ra, dec, lst=None, nu=150e6):
    """
    Return the Jones matrix J.
    @ra: right ascension of the source, in degrees.
    @dec: right ascension of the source, in degrees.
    @nu: frequency of interest, in Hz.

    The default argument comes from the beam that I
    had access to when this was written.
    """
    if lst is None:
        lst = rot.get_lst()
    latitude = np.radians(flux.rot.hera_lat)
    # We want to transform the following into a function parameter
    az, alt = flux.rot.eq_to_topo(ra, dec, latitude, lst, radians=True)

    az = np.array([az])
    alt = np.array([alt])
    
    return flux.stokes.spline_beam_func(nu, alt, az)

def A_matrix_7(ra, dec, nu=150e6):
    """
    Return the Mueller matrix A.
    @ra: right ascension of the source, in degrees.
    @dec: declination of the source, in degrees.
    @nu: frequency of interest, in Hz.

    The default argument comes from the beam that I
    had access to when this was written.
    """
    J = J_matrix_7(ra, dec, nu)
    assert J is not None
    J_outer = np.kron(J, np.conj(J))
    return np.dot(flux.stokes.S, np.dot(J_outer, np.linalg.inv(flux.stokes.S)))
    
def visibility_7(ant1, ant2, source, nu=151e6):
    """
    Visibility integrand evaluated for a single source.

    The most glaring waste of compute is separately calculating
    the values for RA and DEC, although it is not clear to me
    how many clock cycles are actually spent thereon.
    """
    I = source.flux_by_frq[nu / 1e6]
    s = np.array([complex(I), 0, 0, 0])

    ra = np.radians(source.ra_angle)
    dec = np.radians(source.dec_angle)
    r = flux.rot.raddec2lm(ra, dec)

    phi = flux.ant.phase_factor(ant1, ant2, r, nu)
    return np.dot(np.dot(A_matrix_7(ra, dec, nu), s), phi)

In [ ]:
t = 0
while t < 2 * np.pi:
    print(t)
    t += np.pi / 72
    print(J_matrix_7(88, 121, t, flux.catalog.obj_catalog[134]))
    print("\n")
